In [1]:
import wrangle
import acquire
import env

## Bring in the data

In [3]:
python_df = acquire.get_github_python_data()

In [4]:
python_df.head()

,repo_name,url,language,readme_content
0,jackfrued/Python-100-Days,https://github.com/jackfrued/Python-100-Days/b...,Python,Python - 100天从新手到大师作者：骆昊说明：从项目上线到获得8w+星标以来，一直收...
1,Significant-Gravitas/Auto-GPT,https://github.com/Significant-Gravitas/Auto-G...,Python,Auto-GPT: An Autonomous GPT-4 Experiment💡 Get ...
2,shadowsocks/shadowsocks,https://github.com/shadowsocks/shadowsocks/blo...,Python,Removed according to regulations.
3,pallets/flask,https://github.com/pallets/flask/blob/main/REA...,Python,FlaskFlask is a lightweight WSGI web applicati...
4,zero-to-mastery/start-here-guidelines,https://github.com/zero-to-mastery/start-here-...,Python,One rule of this community:We don't care if yo...


In [6]:
type(python_df.readme_content[3])

str

In [6]:
javascript_df = acquire.get_github_java_script_data()

In [7]:
javascript_df.head()

,repo_name,url,language,readme_content
0,mrdoob/three.js,https://github.com/mrdoob/three.js/blob/dev/RE...,JavaScript,three.jsJavaScript 3D libraryThe aim of the pr...
1,academicpages/academicpages.github.io,https://github.com/academicpages/academicpages...,JavaScript,A Github Pages template for academic websites....
2,angular/angular.js,https://github.com/angular/angular.js/blob/mas...,JavaScript,Using AngularJS with the Closure CompilerThe C...
3,nodejs/node,https://github.com/nodejs/node/blob/main/READM...,JavaScript,"Node.jsNode.js is an open-source, cross-platfo..."
4,udacity/frontend-nanodegree-resume,https://github.com/udacity/frontend-nanodegree...,JavaScript,Project DetailsHow do I complete this project?...


## Test the prepare function

In [2]:
train, validate, test = wrangle.wrangle_readmes()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def clean(string):
    """
    This function puts a string in lowercase, normalizes any unicode characters, removes anything that         
    isn't an alphanumeric symbol or single quote.
    """
    # Normalize unicode characters
    string = unicodedata.normalize('NFKD', string)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')
    
    # Remove unwanted characters and put string in lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
            
    return string